# ОН ДАЖЕ С ФИЛЬТРАМИ НЕ СЧИТАЕТ

In [21]:
import os
import csv
import wfdb
import heartpy as hp
import pandas as pd
import numpy as np
from scipy.signal import resample, butter, filtfilt

In [ ]:
age_group = {15:1,16:1,17:1,18:1,19:1,
             20:2,21:2,22:2,23:2,24:2,
             25:3,26:3,27:3,28:3,29:3,
             30:4,31:4,32:4,33:4,34:4,
             35:5,36:5,37:5,38:5,39:5,
             40:6,41:6,42:6,43:6,44:6,
             45:7,46:7,47:7,48:7,49:7,
             50:8,51:8,52:8,53:8,54:8,
             55:9,56:9,57:9,58:9,59:9,
             60:10,61:10,62:10,63:10,64:10,
             65:11,66:11,67:11,68:11,69:11,
             70:12,71:12,72:12,73:12,74:12,
             75:13,76:13,77:13,78:13,79:13,
             80:14,81:14,82:14,83:14,84:14,
             85:15,86:15,87:15,88:15,89:15,
             90:16,91:16,92:16,93:16,94:16,
             95:17,96:17,97:17,98:17,99:17,100:17,
             }

In [ ]:
subjects = pd.read_excel(r"D:\Proga\AML\datasets\lucilacapurro\PPG_Signal_Dataset\subjects_metadata.xlsx",dtype = {"ID":str})
#subjects.dropna(inplace=True)
subjects = subjects[subjects['observations'].isna()]
subjects = subjects.reset_index()
subjects = subjects[["subjectcode","age"]]
subjects["age"] = subjects["age"].map(age_group)
subjects["age"]

In [23]:
record = wfdb.rdrecord('D:\\Proga\\AML\datasets\\physionet.org\\files\\pulse-transit-time-ppg\\1.1.0\\s1_sit')
print(record.__dict__)

{'record_name': 's1_sit', 'n_sig': 18, 'fs': 500, 'counter_freq': None, 'base_counter': None, 'sig_len': 254026, 'base_time': None, 'base_date': None, 'comments': ['<filename>: s1_sit <activity>: sit <gender>: female <height>: 160 <weight>: 50 <age>: 25 <bp_sys_start>: 87 <bp_sys_end>: 87 <bp_dia_start>: 59 <bp_dia_end>: 57 <hr_1_start>: 74 <hr_2_start>: 69 <hr_1_end>: 74 <hr_2_end>: 70 <spo2_start>: 98 <spo2_end>: 98'], 'sig_name': ['ecg', 'pleth_1', 'pleth_2', 'pleth_3', 'pleth_4', 'pleth_5', 'pleth_6', 'lc_1', 'lc_2', 'temp_1', 'temp_2', 'temp_3', 'a_x', 'a_y', 'a_z', 'g_x', 'g_y', 'g_z'], 'p_signal': array([[ 3.07405002e+04,  7.16949830e+04,  7.37368418e+04, ...,
         7.73396744e-03, -4.82016791e-04,  4.57538728e-03],
       [ 3.07200475e+04,  7.16798682e+04,  7.37258545e+04, ...,
         6.81947805e-03,  1.24659515e-03,  4.31670814e-03],
       [ 3.07098211e+04,  7.16798682e+04,  7.37258545e+04, ...,
         6.14014307e-03,  1.91144590e-03,  3.37899626e-03],
       ...,
    

In [38]:
# Определение индекса канала pleth_1
pleth_1_index = record.sig_name.index('pleth_6')

# Извлечение данных pleth_1
pleth_1_data = record.p_signal[:, pleth_1_index]

# Интерполяция данных до 1000 Гц
fs_original = record.fs
fs_new = 1000
pleth_1_data_resampled = resample(pleth_1_data, int(len(pleth_1_data) * fs_new / fs_original), axis=0)

# Обрезка данных до 3 минут (180 секунд)
duration_seconds = 180
samples_to_keep = duration_seconds * fs_new
pleth_1_data_resampled = pleth_1_data_resampled[:samples_to_keep]

# Удаление постоянной составляющей (DC-составляющей)
pleth_1_data_dc_removed = pleth_1_data_resampled - np.mean(pleth_1_data_resampled)

# Фильтрация данных с помощью фильтра Баттерворта
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

lowcut = 0.75
highcut = 5.0
pleth_1_data_filtered = butter_bandpass_filter(pleth_1_data_dc_removed, lowcut, highcut, fs_new)

# Анализ данных PPG с использованием heartpy
wd, m = hp.process(pleth_1_data_filtered, sample_rate=fs_new)

BadSignalWarning: 
----------------
Could not determine best fit for given signal. Please check the source signal.
 Probable causes:
- detected heart rate falls outside of bpmmin<->bpmmax constraints
- no detectable heart rate present in signal
- very noisy signal (consider filtering and scaling)
If you're sure the signal contains heartrate data, consider filtering and/or scaling first.
----------------


In [24]:
fs_dict = {
    'ecg': 500,
    'pleth_1': 500,
    'pleth_2': 500,
    'pleth_3': 500,
    'pleth_4': 500,
    'pleth_5': 500,
    'pleth_6': 500,
    'lc_1': 80,
    'lc_2': 80,
    'temp_1': 10,
    'temp_2': 10,
    'temp_3': 500,
    'a_x': 500,
    'a_y': 500,
    'a_z': 500,
    'g_x': 500,
    'g_y': 500,
    'g_z': 500
}

# Длина сигнала
sig_len = record.sig_len

# Расчет длины каждого сигнала в минутах
durations = {}
for channel, fs in fs_dict.items():
    duration_minutes = sig_len / (fs * 60)
    durations[channel] = duration_minutes

# Вывод результатов
for channel, duration in durations.items():
    print(f"Длина сигнала {channel}: {duration:.2f} минут")

Длина сигнала ecg: 8.47 минут
Длина сигнала pleth_1: 8.47 минут
Длина сигнала pleth_2: 8.47 минут
Длина сигнала pleth_3: 8.47 минут
Длина сигнала pleth_4: 8.47 минут
Длина сигнала pleth_5: 8.47 минут
Длина сигнала pleth_6: 8.47 минут
Длина сигнала lc_1: 52.92 минут
Длина сигнала lc_2: 52.92 минут
Длина сигнала temp_1: 423.38 минут
Длина сигнала temp_2: 423.38 минут
Длина сигнала temp_3: 8.47 минут
Длина сигнала a_x: 8.47 минут
Длина сигнала a_y: 8.47 минут
Длина сигнала a_z: 8.47 минут
Длина сигнала g_x: 8.47 минут
Длина сигнала g_y: 8.47 минут
Длина сигнала g_z: 8.47 минут


In [ ]:
result = []
result_metadata = []
result_signals = {}

target_fs = 1000
five_minute_samples = 300 * target_fs
new_id = 0

for id in ids[:1]:
    new_id += 1 
    try:
        record = wfdb.rdrecord('D:\\Proga\\AML\\datasets\\bruno\\' + id + "\\"+ id+"_PPG")

        #print(record.__dict__)
        #print(record.p_signal.shape)
        #channel_index = record.sig_name.index('NIBP')
        signal = record.p_signal.astype(np.float32)
        fs = record.fs
        print(len(signal[0]))

        if fs != target_fs:
            num_samples = int(len(signal) * target_fs / fs)
            signal = resample(signal, num_samples).astype(np.float32)
            fs = target_fs

        signal = signal[:five_minute_samples]
        
        try:
            wd, m = hp.process(signal[0], fs)
            idx = int(subjects[subjects["ID"] == id].index.to_list()[0])
            age_group = age_groups[idx]
            
            result_metadata.append({
                'id': new_id,
                'age_group': age_group,
                **m
            })
            
            result_signals[new_id] = signal
            
            print((new_id, id, age_group, signal, m))

        except hp.exceptions.BadSignalWarning as e:
            result_metadata.append({
                'id': new_id,
                'age_group': None,
                'error': "BadSignalWarning"
            })
            print((new_id, id, None, None, None, "BadSignalWarning"))

    except FileNotFoundError as e:
        result_metadata.append({
            'id': new_id,
            'age_group': None,
            'error': "FileNotFound"
        })
        print((new_id, id, None, None, None, "FileNotFound"))

    print(f"Process end: {id}")